## **项目理解**


### **1 项目总结构**

具体代码目录：

```css
📦 根目录
├── 📂 agent
│   ├── 📂 algorithm
│       └── 📄 __init__.py
│       └── 📄 algorithm.py
│   ├── 📂 conf
│       └── 📄 __init__.py
│       └── 📄 conf.py
│       └── 📄 train_env_conf.toml
│   ├── 📂 feature
│       └── 📄 __init__.py
│       └── 📄 definition.py
│       └── 📄 preprocessor.py
│   ├── 📂 model
│       └── 📄 __init__.py
│       └── 📄 model.py
│   ├── 📂 workflow
│       └── 📄 __init__.py
│       └── 📄 train_workflow.py
│   ├── 📄 __init__.py
│   └── 📄 agent.py
├── 📂 conf
│   ├── 📄 __init__.py
│   ├── 📄 configure_app.toml
├── 📂 log



└── 📄 train_test.py
```

具体作用如下：

![总描述](image/5.png)

![agent目录](image/6.png)

![conf目录](image/7.png)


### **2 环境系统总述**

有两个接口函数，注意这两个接口函数的实现以及具体的调用都是由运行时注入并通过`RPC/HTTP`等方式调用远端服务的，故只需要了解其的传入参数以及返回参数即可，如下：


#### **2.1 接口函数**

-  `env.reset(usr_conf: dict) → (obs, state)`根据`train_env_conf.toml`中的参数初始化环境，返回初始观测与全局状态
-  `env.step(act: int) → (frame_no, obs, score, terminated, truncated, extra_info)`接受动作(0: UP, 1: DOWN, 2: LEFT, 3: RIGHT)，执行一步环境转移，返回当前帧编号，观测，即时得分，是否结束，是否异常中断及额外信息

#### **2.2 返回数据结构**

- `obs`：包含`feature`一维特征向量与`legal_act`合法动作列表
- `extra_info`，包括：
    - `frame_state`当前环境内部状态
    - `game_info`：`pos_x/pos_z`、`step_no、treasure_count`、`treasure_status`、`local_view`（5×5局部视野展平）及 `location_memory`（64×64全局记忆展平）
    - `score_info`：即时与累计得分


### **3 智能体执行流程**

#### **3.1 环境模型**

项目提供了一个预先计算好的状态转移字典F(存储于`conf/map_data/F_level_1.json`)，描述了每个状态下执行个动作的结果。`F`的结构是一个嵌套字典：键为状态编号，值为该状态下各动作的结果；每个动作结果包括下一个状态、获得的奖励以及是否结束标志。

#### **3.2 算法流程**

该模块实现了值迭代和策略迭代两种方法，默认采用值迭代

- 算法初始化时，初始化一个随机均匀策略表
```python
self.agent_policy = np.ones([self.state_size, self.action_size]) / self.action_size
```

- 进入算法具体实现：

值迭代核心函数：

$$ V_{k+1}\left( s \right) =\underset{a\in \mathcal{A}}{\max}\left[ R\left( s,a \right) +\gamma P\left( s^{\prime}|s,a \right) V_k\left( s^{\prime} \right) \right] $$

在确定的环境下，转移概率$ P\left( s^{\prime}|s,a \right) $为$1$或$0$，公式化简为：

$$ V_{k+1}(s) = \max_{a} \left[ R(s,a) + \gamma V_k(s^{\prime}) \right]$$

当所有状态更新后的最大差值$\max _s|V_{k+1}\left( s \right) -V_k\left( s \right) |$小于阈值$\theta$，认为收敛，取$V^{*}=V_{k+1}$。此时最优策略可由：

$$\pi^{\star}(s) = \arg\max_{a} \left[ R(s,a) + \gamma V^{\star}(s^{\prime}) \right]$$

直接导出
